In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
from tqdm import tqdm
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

# Data Import

Supervised NLP requires a pre-labelled dataset for training and testing, and is generally interested in categorizing text in various ways. In this case, we are going to try to predict whether a sentence comes from _Alice in Wonderland_ by Lewis Carroll or _Persuasion_ by Jane Austen. We can use any of the supervised models we've covered previously, as long as they allow categorical outcomes. In this case, we'll try Random Forests, SVM, and KNN.

Our feature-generation approach will be something called _BoW_, or _Bag of Words_. BoW is quite simple: For each sentence, we count how many times each word appears. We will then use those counts as features.  

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [3]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [4]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [5]:
len(sentences[0][0])

67

In [6]:
sentences[0][0]

Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, 'and what is the use of a book,' thought Alice 'without pictures or conversation?'

Time to bag some words!  Since spaCy has already tokenized and labelled our data, we can move directly to recording how often various words occur.  We will exclude stopwords and punctuation.  In addition, in an attempt to keep our feature space from exploding, we will work with lemmas (root words) rather than the raw text terms, and we'll only use the 2000 most common words for each text.

In [7]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words, chunkstart, chunkstop):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    print(df['text_sentence'][chunkstart:chunkstop])
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in tqdm(enumerate(df['text_sentence'][chunkstart:chunkstop])):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df.iloc[np.arange(chunkstart, chunkstop), :]

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [8]:
len(alicewords)

1889

In [9]:
len(persuasionwords)

2000

In [10]:
len(common_words)

3062

In [11]:
# Create our data frame with features. This can take a while to run.
try:
    alice500 = pd.read_csv('word_counts500.csv', index_col=0)
    print(len(alice500), len(alice500.columns))
except FileNotFoundError:
    word_counts500 = bow_features(sentences, common_words, 0, 500)
    word_counts500.to_csv('word_counts500.csv')

500 3064


In [12]:
try:
    alice1000 = pd.read_csv('word_counts1000.csv', index_col=0)
    print(len(alice1000), len(alice1000.columns))
except FileNotFoundError:
    word_counts1000 = bow_features(sentences, common_words, 500, 1000)
    word_counts1000.to_csv('word_counts1000.csv')

500 3064


In [13]:
try:
    alice1500 = pd.read_csv('word_counts1500.csv', index_col=0)
    print(len(alice1500))
except FileNotFoundError:
    word_counts1500 = bow_features(sentences, common_words, 1000, 1500)
    word_counts1500.to_csv('word_counts1500.csv')

500


In [14]:
try:
    alice2000 = pd.read_csv('word_counts2000.csv', index_col=0)
    print(len(alice2000))
except FileNotFoundError:
    word_counts2000 = bow_features(sentences, common_words, 1500, 2000)
    word_counts2000.to_csv('word_counts2000.csv')

500


In [15]:
try:
    alice2500 = pd.read_csv('word_counts2500.csv', index_col=0)
    print(len(alice2500))
except FileNotFoundError:
    word_counts2500 = bow_features(sentences, common_words, 2000, 2500)
    word_counts2500.to_csv('word_counts2500.csv')

500


In [16]:
try:
    alice3000 = pd.read_csv('word_counts3000.csv', index_col=0)
    print(len(alice3000))
except FileNotFoundError:
    word_counts3000 = bow_features(sentences, common_words, 2500, 3000)
    word_counts3000.to_csv('word_counts3000.csv')

500


In [17]:
try:
    alice3500 = pd.read_csv('word_counts3500.csv', index_col=0)
    print(len(alice3500))
except FileNotFoundError:
    word_counts3500 = bow_features(sentences, common_words, 3000, 3500)
    word_counts3500.to_csv('word_counts3500.csv')

500


In [18]:
try:
    alice4000 = pd.read_csv('word_counts4000.csv', index_col=0)
    print(len(alice4000))
except FileNotFoundError:
    word_counts4000 = bow_features(sentences, common_words, 3500, 4000)
    word_counts4000.to_csv('word_counts4000.csv')

500


In [19]:
try:
    alice4500 = pd.read_csv('word_counts4500.csv', index_col=0)
    print(len(alice4500))
except FileNotFoundError:
    word_counts4500 = bow_features(sentences, common_words, 4000, 4500)
    word_counts4500.to_csv('word_counts4500.csv')

500


In [20]:
# Create our data frame with features. This can take a while to run.
try:
    alice5000 = pd.read_csv('word_counts5000.csv', index_col=0)
    print(len(alice5000))
except FileNotFoundError:
    word_counts5000 = bow_features(sentences, common_words, 4500, 5000)
    word_counts5000.to_csv('word_counts5000.csv')

500


In [21]:
try:
    alice5318 = pd.read_csv('word_counts5318.csv', index_col=0)
    print(len(alice5318))
except FileNotFoundError:
    word_counts5318 = bow_features(sentences, common_words, 5000, 5318)
    word_counts5318.to_csv('word_counts5318.csv')

318


In [22]:
aliceframes = [alice500, alice1000, alice1500, alice2000, alice2500, alice3000, alice3500, alice4000, alice4500,
          alice5000, alice5318]

word_counts = pd.concat(aliceframes)

In [23]:
word_counts.head()

,condescend,brandy,fold,careful,objection,pennyworth,as,rent,mischief,naval,...,calmly,protest,improvement,desert,righthand,antipathies,employment,poky,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Alice was beginning to get very tired of sitti...,Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,So she was considering in her own mind (as wel...,Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,There was nothing so VERY remarkable in that; ...,Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Oh dear!,Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,I shall be late!',Carroll


# Modeling

## Trying out BoW

Now let's give the bag of words features a whirl by trying a random forest.

In [24]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

y_pred_rfc = rfc.predict(X_test)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

print(pd.crosstab(y_test, y_pred_rfc, margins=True))

print('\n\n')
y_pred_all = rfc.predict(X)

print(pd.crosstab(Y, y_pred_all, margins=True))

Training set score: 0.7764890282131661

Test set score: 0.7725563909774437
col_0        Austen  Carroll   All
text_source                       
Austen         1472        0  1472
Carroll         484      172   656
All            1956      172  2128



col_0        Austen  Carroll   All
text_source                       
Austen         3649        0  3649
Carroll        1197      472  1669
All            4846      472  5318


None of the Austen lines were predicted to be Carroll, but most of the Carroll lines were predicted to be Austen.

In [25]:
cvscore = cross_val_score(rfc, X, Y, cv=5, n_jobs=-1)

In [26]:
cvscore

array([0.95958647, 0.81672932, 0.68609023, 0.68609023, 0.68644068])

Holy overfitting, Batman! Overfitting is a known problem when using bag of words, since it basically involves throwing a massive number of features at a model – some of those features (in this case, word frequencies) will capture noise in the training set. Since overfitting is also a known problem with Random Forests, the divergence between training score and test score is expected.


## BoW with Logistic Regression

Let's try a technique with some protection against overfitting due to extraneous features – logistic regression with ridge regularization (from ridge regression, also called L2 regularization).

In [27]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

y_pred_lr = lr.predict(X_test)

print(pd.crosstab(y_test, y_pred_lr, margins=True))

print('\n\n')
y_pred_all = lr.predict(X)

print(pd.crosstab(Y, y_pred_all, margins=True))

(3190, 3062) (3190,)
Training set score: 0.7724137931034483

Test set score: 0.768796992481203
col_0        Austen  Carroll   All
text_source                       
Austen         1472        0  1472
Carroll         492      164   656
All            1964      164  2128



col_0        Austen  Carroll   All
text_source                       
Austen         3649        0  3649
Carroll        1218      451  1669
All            4867      451  5318


In [28]:
"""cvscore = cross_val_score(lr, X, Y, cv=5, n_jobs=-1)"""

'cvscore = cross_val_score(lr, X, Y, cv=5, n_jobs=-1)'

In [29]:
"""cvscore"""

'cvscore'

Logistic regression performs a bit better than the random forest.

# BoW with Gradient Boosting

And finally, let's see what gradient boosting can do:

In [30]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

y_pred_clf = clf.predict(X_test)

pd.crosstab(y_test, y_pred_clf, margins=True)

print('\n\n')
y_pred_all = clf.predict(X)

print(pd.crosstab(Y, y_pred_all, margins=True))

Training set score: 0.7730407523510971

Test set score: 0.7631578947368421



col_0        Austen  Carroll   All
text_source                       
Austen         3649        0  3649
Carroll        1228      441  1669
All            4877      441  5318


In [31]:
cvscore = cross_val_score(clf, X, Y, cv=5, n_jobs=-1)

In [32]:
cvscore

array([0.93515038, 0.79981203, 0.68609023, 0.68609023, 0.68644068])

In [33]:
word_counts.text_source.value_counts()

Austen     3649
Carroll    1669
Name: text_source, dtype: int64

In [34]:
word_counts['text_source'] = word_counts['text_source'].replace({'Austen': 0, 'Carroll': 1})

In [35]:
word_counts.text_source.value_counts()

0    3649
1    1669
Name: text_source, dtype: int64

Looks like logistic regression is the winner, but there's room for improvement.

# Same model, new inputs

What if we feed the model a different novel by Jane Austen, like _Emma_?  Will it be able to distinguish Austen from Carroll with the same level of accuracy if we insert a different sample of Austen's writing?

First, we need to process _Emma_ the same way we processed the other data, and combine it with the Alice data:

In [36]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [37]:
len(emma)

876200

In [38]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [39]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]

In [40]:
emma_sentences = pd.DataFrame(emma_sents)

In [41]:
len(emma_sentences)

1669

In [42]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.

try:
    emma500 = pd.read_csv('emma_bow500.csv', index_col=0)
    print(len(emma500))
except FileNotFoundError:
    emma_bow500 = bow_features(emma_sentences, common_words, 0, 500)
    emma_bow500.to_csv('emmabow500.csv')

500


In [43]:
try:
    emma1000 = pd.read_csv('emma_bow1000.csv', index_col=0)
    print(len(emma1000))
except FileNotFoundError:
    emma_bow1000 = bow_features(emma_sentences, common_words, 500, 1000)
    emma_bow1000.to_csv('emma_bow1000.csv')

500


In [44]:
try:
    emma1669 = pd.read_csv('emma_bow1669.csv', index_col=0)
    print(len(emma1669))
except FileNotFoundError:
    emma_bow1669 = bow_features(emma_sentences, common_words,1000, 1669)
    emma_bow1669.to_csv('emma_bow1669.csv')

669


In [45]:
emma_chunks = [emma500, emma1000, emma1669]
emma_bow = pd.concat(emma_chunks)
emma_bow

,condescend,brandy,fold,careful,objection,pennyworth,as,rent,mischief,naval,...,calmly,protest,improvement,desert,righthand,antipathies,employment,poky,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"Emma Woodhouse, handsome, clever, and rich, wi...",Austen
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,She was the youngest of the two daughters of a...,Austen
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Her mother had died too long ago for her to ha...,Austen
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Sixteen years had Miss Taylor been in Mr. Wood...,Austen
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Between _them,Austen
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,_,Austen
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,it was more the intimacy of sisters.,Austen
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Even before Miss Taylor had ceased to hold the...,Austen
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"The real evils, indeed, of Emma's situation we...",Austen
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"The danger, however, was at present so unperce...",Austen


In [46]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)), axis=0)

y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted, margins=True)


Test set score: 0.529082774049217


col_0,Austen,Carroll,All
row_0,,,
Austen,1308,361,1669
Carroll,902,111,1013
All,2210,472,2682


Well look at that!  NLP approaches are generally effective on the same type of material as they were trained on. It looks like this model is actually able to differentiate multiple works by Austen from Alice in Wonderland.  Now the question is whether the model is very good at identifying Austen, or very good at identifying Alice in Wonderland, or both...

# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93%.  See what you can do to improve performance.  Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.  

In [47]:
svc = SVC()
train = svc.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', svc.score(X_train, y_train))
print('\nTest set score:', svc.score(X_test, y_test))

(3190, 3062) (3190,)
Training set score: 0.6824451410658308

Test set score: 0.6917293233082706


In [48]:
print('\nTest set score:', svc.score(X_Emma_test, y_Emma_test))
svc_Emma_predicted = svc.predict(X_Emma_test)
pd.crosstab(y_Emma_test, svc_Emma_predicted, margins=True)


Test set score: 0.622296793437733


col_0,Austen,All
row_0,,
Austen,1669,1669
Carroll,1013,1013
All,2682,2682


# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

In [49]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [50]:
# Clean the Melville data.
moby = gutenberg.raw('melville-moby_dick.txt')
moby = re.sub(r'VOLUME \w+', '', moby)
moby = re.sub(r'CHAPTER \w+', '', moby)
moby = text_cleaner(moby)
len(moby)

1207515

In [51]:
"""# Parse our cleaned data.
moby_doc = nlp(moby)"""

'# Parse our cleaned data.\nmoby_doc = nlp(moby)'

In [52]:
# Clean the Blake data.
blake = gutenberg.raw('blake-poems.txt')
blake = re.sub(r'VOLUME \w+', '', blake)
blake = re.sub(r'CHAPTER \w+', '', blake)
blake = text_cleaner(blake)
len(blake)

36051

In [53]:
# Parse our cleaned data.
blake_doc = nlp(blake)

In [54]:
# Group into sentences.
blake_sents = [[sent, "Blake"] for sent in blake_doc.sents]

# blake is quite long, let's cut it down to the same length as Alice.
blake_sents = blake_sents[0:len(alice_sents)]

In [55]:
blake_sentences = pd.DataFrame(blake_sents)

In [56]:
len(blake_sentences)

498

In [57]:
blake_sentences.head()

,0,1
0,"(SONGS, OF, INNOCENCE, AND, OF, EXPERIENCE, an...",Blake
1,"(INTRODUCTION, Piping, down, the, valleys, wil...",Blake
2,"(So, I, piped, with, merry, cheer, ., "")",Blake
3,"(Piper, ,, pipe, that, song, again)",Blake
4,"(;, "", So, I, piped, :, he, wept, to, hear, ., "")",Blake


In [58]:
try:
    blake_bow = pd.read_csv('blake.csv', index_col=0)
    print(len(blake_bow))
except FileNotFoundError:
    blake_bow = bow_features(blake_sentences, common_words, 0, len(blake_sentences))
    blake_bow.to_csv('blake.csv')

498


In [59]:
word_counts.to_csv('word_counts.csv')

In [60]:
emma_bow.to_csv('emma_counts.csv')